In [ ]:
# import lasio_utilitites as lasutil
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
#from math import isclose

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# las files
lasfiles = lasutil.loader_lasio_multiprocess("F:\.repos\DarkVision\miscelaneous\data", outdf=False)
lasfiles.keys()

In [ ]:
gg_df = lasfiles['ConocoPhillips_Stolte_Unit_A1_2D_Log_2022-04-03'].df()

In [ ]:
gg_df.to_pickle('..\data\gg_df_gz.pkl', compression='gzip')

In [ ]:
gg_df = pd.read_pickle('..\data\gg_df_gz.pkl', compression='gzip')

In [ ]:
gg_df.shape

In [ ]:
#Resampling
#df = df.reindex(df.index.union(resampling))

#Interpolation technique to use. One of:

#'linear': Ignore the index and treat the values as equally spaced. This is the only method supported on MultiIndexes.
#'time': Works on daily and higher resolution data to interpolate given length of interval.
#'index', 'values': use the actual numerical values of the index.
#'pad': Fill in NaNs using existing values.
#'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial': Passed to scipy.interpolate.interp1d. These methods use the numerical values of the index. Both 'polynomial' and 'spline' require that you also specify an order (int), e.g. df.interpolate(method='polynomial', order=5).
#'krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima': Wrappers around the SciPy interpolation methods of similar names. See Notes.
#'from_derivatives': Refers to scipy.interpolate.BPoly.from_derivatives which replaces 'piecewise_polynomial' interpolation method in scipy 0.18.
# new_index = np.linspace(gg_df.index.min(),gg_df.index.max(),gg_df.shape[0]//10)
new_index = np.arange(gg_df.index.min(),gg_df.index.max(),1)
gg_df_resampled = gg_df.reindex(gg_df.index.union(new_index)).interpolate('values').loc[new_index]

In [ ]:
gg_df_resampled.shape

In [ ]:
from tempfile import template
from numpy import spacing


def build_log_graph(df):
    logplot = make_subplots(rows=1, cols=5, shared_yaxes=True, horizontal_spacing=0.01, specs=[[{'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}, {'secondary_y': True}]], print_grid=True)
    # for i in range(len(logs)):

    #     if i == 1:
    #         logplot.add_trace(go.Scatter(x=dataframes[plot_chosen][logs[i]], y=dataframes[plot_chosen]['DEPTH_MD'],
    #                                      name=logs[i], line_color=colors[i]), row=1, col=log_cols[i])
    #         logplot.update_xaxes(type='log', row=1, col=log_cols[i], title_text=logs[i],
    #                              tickfont_size=12, linecolor='#585858')

    #     else:
    #         logplot.add_trace(go.Scatter(x=dataframes[plot_chosen][logs[i]], y=dataframes[plot_chosen]['DEPTH_MD'],
    #                                      name=logs[i], line_color=colors[i]), row=1, col=log_cols[i])
    #         logplot.update_xaxes(col=log_cols[i], title_text=logs[i], linecolor='#585858')

    logplot.add_trace(go.Scatter(x=df.OVALITY, y=df.index, name=df.OVALITY.name), row=1, col=1 ,secondary_y=False)
    logplot.add_trace(go.Scatter(x=df.ECCENTRICITY, y=df.index, name=df.ECCENTRICITY.name), row=1, col=1, secondary_y=False)

    logplot.layout.xaxis2.update({'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    logplot.add_trace(go.Scatter(x=df.MINID, y=df.index, name=df.MINID.name), row=1, col=3)
    logplot.add_trace(go.Scatter(x=df.MEANID, y=df.index, name=df.MEANID.name), row=1, col=3)
    logplot.add_trace(go.Scatter(x=df.MAXID, y=df.index, name=df.MAXID.name), row=1, col=3)
    logplot.update_xaxes(row=1, col=3, title_text='ID')
    
    logplot.add_trace(go.Scatter(x=df.MINPEN, y=df.index, name=df.MINPEN.name), row=1, col=4)
    logplot.add_trace(go.Scatter(x=df.MEANPEN, y=df.index, name=df.MEANPEN.name), row=1, col=4)
    logplot.add_trace(go.Scatter(x=df.MAXPEN, y=df.index, name=df.MAXPEN.name), row=1, col=4)
    logplot.update_xaxes(row=1, col=4, title_text='Penetration', range=[-0.1, 0.1])

    logplot.add_trace(go.Heatmap(z=df[[fing for fing in df.columns if 'FING' in fing]], y=df.index, colorscale='greys', name='Image', x=np.arange(0,361, 1.879901)), row=1, col=5)
    logplot.update_xaxes(row=1, col=5, title_text='Radial Measurement', range=[0, 360])

    logplot.update_xaxes(showline=True, linewidth=2, mirror=True, ticks='inside', tickangle=0)
    logplot.update_yaxes(showline=True, linewidth=2,
                         mirror=True, ticks='outside')
    logplot.update_yaxes(row=1, col=1, autorange='reversed')
    logplot.update_layout(height=750, width=1300, showlegend=True, legend_orientation='h', yaxis_title='Depth (m)', template='plotly_dark')

    return logplot

build_log_graph(gg_df_resampled)

In [ ]:
gg_df['CLASSIFICATION'].where((gg_df['CLASSIFICATION'] < 1.0) & (gg_df['CLASSIFICATION'] > -999.25), 0.0, inplace=True)

In [ ]:
# gg_df['MINIDPHASE_FLAG'] = gg_df['MINIDPHASE'].loc[gg_df['OVALITY'] > 2]
# gg_df['MAXIDPHASE _FLAG'] = gg_df['MAXIDPHASE'].loc[gg_df['OVALITY'] > 2]

# gg_df['MINIDPHASE'] = pd.Series(gg_df['MINIDPHASE'].rolling(window=32, min_periods=1, center=True).mean())
# gg_df['MAXIDPHASE'] = pd.Series(gg_df['MAXIDPHASE'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['OVALITY'] = pd.Series(gg_df['OVALITY'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['ECCENTRICITY'] = pd.Series(gg_df['ECCENTRICITY'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MINID'] = pd.Series(gg_df['MINID'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MAXID'] = pd.Series(gg_df['MAXID'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MEANID'] = pd.Series(gg_df['MEANID'].rolling(window=32, min_periods=1, center=True).mean())

# gg_df['MINIDPHASE'] = gg_df['MINIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXIDPHASE'] = gg_df['MAXIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['OVALITY'] = gg_df['OVALITY'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['ECCENTRICITY'] = gg_df['ECCENTRICITY'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MINID'] = gg_df['MINID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXID'] = gg_df['MAXID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MEANID'] = gg_df['MEANID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MINIDPHASE'] = gg_df['MINIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXIDPHASE'] = gg_df['MAXIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]


# gg_df['MINIDPHASE'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXIDPHASE'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MEANID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXPENJOINT'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['CLASSIFICATION'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MEANPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['ECCENTRICITY'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['OVALITY'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)

In [ ]:
gg_df['MEANID'] = gg_df['MEANID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]

In [ ]:
gg_df['CLASSIFICATION'].plot(template='plotly_dark')

In [ ]:
gg_df['MEANID'].plot(template='plotly_dark')

In [ ]:
window_size=64
threshold=100
min_periods=1
df = gg_df.copy()

for colname in df.columns:
    smothy = pd.Series(df[colname].rolling(window=window_size, min_periods=min_periods, center=True).mean())
    difference = pd.DataFrame.abs(df[colname] - smothy)
    outliers = difference > threshold
    #df[colname].where(outliers, smothy, inplace=True)
    df[colname] = pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])


In [ ]:
def smothy_data(df_org, window_size=64, threshold=100, min_periods=1):
    """
    Smothy data using rolling mean and thresholding
    """
    df = df_org.copy()
    for colname in df.columns:
        smothy = pd.Series(df[colname].rolling(window=window_size, min_periods=min_periods, center=True).mean())
        difference = pd.DataFrame.abs(df[colname] - smothy)
        outliers = difference > threshold
        #df[colname].where(outliers, smothy, inplace=True)
        df[colname] = pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])
    return df

    #return pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])

gg_df = smothy_data(gg_df, window_size=32, threshold=0.01, min_periods=1)

In [ ]:
lasfiles['gg_2_curves'].set_data(gg_df)
lasfiles['gg_2_curves'].write('gg_2_curves_FIXED.las', version=2.0)